In [238]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale

In [239]:
def readDataset(path1, path2):
  magicset = pd.read_csv(path1)
  letterset = pd.read_csv(path2)
  return magicset, letterset

In [240]:
def manageDataset(magicset, letterset):
  letterset = letterset.rename({'letter': 'class'}, axis=1)
  magicset = magicset.sample(frac=1,random_state=42).reset_index(drop=True)
  magicset = magicset.drop(['Unnamed: 0'],axis = 1)
  letterset = letterset.replace(['A','B','C','D','E','F','G','H','I','J','K','L','M'],1)
  letterset = letterset.replace(['N','O','P','Q','R','S','T','U','V','W','X','Y','Z'],-1)
  return magicset, letterset


In [241]:
magicset, letterset = readDataset('/content/Magic.csv', '/content/letter-recognition.csv')
magicset, letterset = manageDataset(magicset, letterset)

In [242]:
letterset

,class,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,-1,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,1,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,-1,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,1,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,1,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,-1,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,-1,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [243]:
#printing letters in order
m = len(magicset['class'])

def orderLetters(m, magicset):
  zeroCount = 0
  for i in range(m):
    if ord(magicset['class'][i]) == ord('g'):
      magicset['class'][i] = 1
      zeroCount += 1
    else:
      if ord(magicset['class'][i]) == ord('h'):
        magicset['class'][i] = -1
  print(zeroCount)
  return zeroCount, magicset

zeroCount, magicset = orderLetters(m, magicset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


12332


In [244]:
def splitData(magicset, letterset):
  magicTrain=magicset.sample(frac=0.666667)
  letterTrain=letterset.sample(frac=0.666667)
  magicTest = magicset.drop(magicTrain.index)
  letterTest= letterset.drop(letterTrain.index)
  return magicTrain, letterTrain, magicTest, letterTest

In [245]:
magicTrain, letterTrain, magicTest, letterTest = splitData(magicset, letterset)
print(magicTrain.shape)
magicTrain

(12680, 11)


,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
8980,12.8581,9.5270,2.3570,0.8176,0.5868,10.7219,9.3931,8.6736,14.5600,39.9414,-1
12433,133.9489,38.3972,3.0833,0.2517,0.1421,89.9043,128.2511,26.3516,43.5612,32.7130,-1
4213,28.7453,10.6247,2.5977,0.5278,0.3371,18.9851,20.6578,6.0474,15.0330,154.9800,1
7327,272.0630,20.1242,2.5563,0.4556,0.2319,-349.7570,203.8630,-13.8784,62.3504,184.0600,1
4478,65.7368,32.8299,3.3320,0.1364,0.0710,83.0276,49.2726,-24.2423,11.6143,153.8250,1
...,...,...,...,...,...,...,...,...,...,...,...
3443,30.7003,13.6684,2.6580,0.4264,0.2187,-2.2110,12.0845,4.3089,2.2951,178.5270,1
5807,53.5632,41.9902,2.8442,0.2792,0.1539,-55.7825,-45.0453,-15.8210,61.5552,124.9570,1
1430,30.8978,13.8836,2.8488,0.4391,0.2812,14.2610,29.9884,-6.0247,14.4319,63.7038,1
15922,18.3486,12.6965,2.5484,0.5714,0.3182,-3.2207,11.7192,-9.0724,1.6196,253.1820,-1


In [246]:
print(letterTrain.shape)
letterTrain
# letterTrain=letterset.rename({'letter': 'class'}, axis=1)

(13333, 17)


,class,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
9961,1,3,11,6,8,4,11,4,2,3,9,2,9,3,7,3,8
17976,-1,6,11,5,6,3,10,3,4,6,10,3,9,3,8,6,13
18856,-1,3,5,4,6,4,7,8,5,2,7,8,11,2,9,5,8
5878,-1,7,12,6,7,4,7,10,3,5,13,5,3,3,11,5,6
7015,1,4,10,6,8,5,8,7,5,9,7,6,8,3,8,6,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,-1,4,5,5,7,3,7,7,15,2,4,6,8,6,8,0,8
16590,-1,3,6,4,4,4,8,6,6,3,7,5,7,4,6,6,7
5407,1,2,9,3,7,3,8,7,0,7,7,6,8,0,8,3,7
3646,-1,2,1,3,3,2,7,8,5,9,6,6,9,1,9,7,8


In [247]:
def optDataSvm(Data, testData):
    X_train = Data.drop(['class'],axis = 1)
    y_train = Data['class']
    y_train = y_train.astype('int')
    X_test = testData.drop(['class'],axis = 1)
    y_test = testData['class']
    y_test=y_test.astype('int')
    return X_train, y_train, X_test, y_test

In [248]:
def svm_random(trainData,testData):
  prob2=0;
  linear_model = SVC(kernel='linear')
  accuracyList=[]
  while prob2<20:
    Data_random=trainData.sample(n=500)
    X_train, y_train, X_test, y_test = optDataSvm(Data_random, testData)
    linear_model.fit(X_train,y_train)
    y_pred = linear_model.predict(X_test)
    prob2+=1
    accuracy=metrics.accuracy_score(y_true = y_test,y_pred = y_pred)
    accuracyList.append(1-accuracy)
  return 100*np.mean(accuracyList),100*np.std(accuracyList)

In [249]:
def hingeLoss(f,z):
  if f*z>1:
    return 0
  else:
    return (1-f*z)

In [250]:
def appSample(id, prob, yt, yst, sample, kvalue, zstl, kvalmax):
  if prob==1 and yt==-1 and yst==-1:
    sample = sample.append(pd.DataFrame(zstl,index=[id+1],columns=sample.columns))
  elif prob==1 and yt==1 and yst==1:
    sample = sample.append(pd.DataFrame(zstl,index=[id+1],columns=sample.columns))
  elif (prob==1 and yt*yst==-1) or prob<0:
    sample = sample.append(pd.DataFrame(zstl,index=[id+1],columns=sample.columns))
  elif prob>1:
    sample = sample.append(pd.DataFrame(zstl,index=[id+1],columns=sample.columns))
  elif kvalue==kvalmax:
    sample = sample.append(pd.DataFrame(zstl,index=[id+1],columns=sample.columns))
    kvalue=0
  else:
    kvalue+=1
  return kvalue, sample

In [251]:
import math
def markovsampling(dataFrame):
  linear_model = SVC(kernel='linear')
  
  
  T=5
  N=len(dataFrame)
  rsample = dataFrame[0:0]
  t=0
  while t<T:
    sample = dataFrame[0:0]
    dtr=dataFrame.copy()
    X_train = dtr.drop(['class'],axis = 1)
    y_train = dtr['class']
    y_train=y_train.astype('int')
    linear_model.fit(X_train,y_train)
    kvalmax, kvalue, m0, m1 = 5, 0, 0, 0
    zt=dtr.sample(n=1)
    ztl=zt.to_numpy()
    id=len(sample)
    sample = sample.append(pd.DataFrame(ztl,index=[id+1],columns=sample.columns))
    dtr=dtr.drop(zt.index)
    yl=zt['class'].to_numpy()
    yt=yl[0]
    if yt==-1:
      m0+=1
    else:
      m1+=1
    ydt=linear_model.predict(zt.drop(['class'],axis = 1))
    
    i=0
    while i<N and len(dtr)>0:
      id=len(sample)
      zst=dtr.sample(n=1)
      zstl=zst.to_numpy()
      dtr=dtr.drop(zst.index)
      ysl=zst['class'].to_numpy()
      yst=ysl[0]
      ydst=linear_model.predict(zst.drop(['class'],axis = 1))
      prob=math.exp(-hingeLoss(ydst,ydst))/math.exp(-hingeLoss(ydt,yt))
      kvalue , sample = appSample(id, prob, yt, yst, sample, kvalue, zstl, kvalmax)
      zt=zst
      if yst==-1:
       m0+=1
      else:
       m1+=1
    i=i+1
    # print(len(sample))
    X_train = sample.drop(['class'],axis = 1)
    y_train = sample['class']
    y_train=y_train.astype('int')
    linear_model.fit(X_train,y_train)
    rsample=sample
    t=t+1
  return rsample
    


In [252]:
def svm_markov(trainData,testData):
  prob2=0;
  linear_model = SVC(kernel='linear')
  accuracyList=[]
  
  while prob2<20:
    dtr=trainData.sample(n=500)
    dtr.reset_index(drop=True, inplace=True)  
    Data_markov=markovsampling(dtr)
    # print(Data_markov.shape)
    X_train, y_train, X_test, y_test = optDataSvm(Data_markov, testData)
    linear_model.fit(X_train,y_train)
    y_pred = linear_model.predict(X_test)
    accuracy=metrics.accuracy_score(y_true = y_test,y_pred = y_pred)
    accuracyList.append(1-accuracy)
    prob2+=1

  return 100*np.mean(accuracyList),100*np.std(accuracyList)

In [253]:
print("Error on letter dataset with random sampling",svm_random(letterTrain,letterTest))

Error on letter dataset with random sampling (27.781610919454025, 0.7271558451362401)


In [254]:
print("Error on letter dataset with boosting through markov sampling",svm_markov(letterTrain,letterTest))

Error on letter dataset with boosting through markov sampling (27.980350982450876, 0.7615495323884576)


In [255]:
print("Error on magic dataset with random sampling",svm_random(magicTrain,magicTest))

Error on magic dataset with random sampling (21.063091482649842, 0.3649775471143405)


In [256]:
print("Error on magic dataset with boosting through markov sampling",svm_markov(magicTrain,magicTest)

Error on magic dataset with boosting through markov sampling (21.078864353312305, 0.3236476221749053)
